# Анализ эффективности удержания

В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.

Данные находястя в файле **churn_analysis.csv**

Измерены следующие признаки:

* state — штат США
* account_length — длительность использования аккаунта
* area_code — деление пользователей на псевдорегионы, использующееся в телекоме
* intl_plan — подключена ли у пользователя услуга международного общения
* vmail_plan — подключена ли у пользователя услуга голосовых сообщений
* vmail_message — количество голосых сообщений, который пользователь отправил / принял
* day_calls — сколько пользователь совершил дневных звонков
* day_mins — сколько пользователь проговорил минут в течение дня
* day_charge — сколько пользователь заплатил за свою дневную активность
* eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
* night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
* intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
* custserv_calls — сколько раз пользователь позвонил в службу поддержки
* treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
* mes_estim — оценка интенсивности пользования интернет мессенджерами
* churn — результат оттока: перестал ли абонент пользоваться услугами оператора

## Task 1
Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции **scipy.stats.chi2_contingency(subtable, correction=False)**

Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько достигаемых уровней значимости оказались меньше, чем $\alpha=0.05$?

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from itertools import combinations
%matplotlib inline

In [3]:
data = pd.read_csv('datasets/churn_analysis.csv', index_col=0)
data.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [4]:
data.replace({'False.': False, 'True.': True}, inplace=True)

In [5]:
data.churn.value_counts()

False    2850
True      483
Name: churn, dtype: int64

In [6]:
state_churn_table_1 = pd.crosstab(data[data.treatment == 1].state, data[data.treatment == 1].churn)
state_churn_table_1.head()

churn,False,True
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5


In [7]:
combs = combinations(state_churn_table_1.index, 2)
n_pvals = 0
for comb in combs:
    subtable = state_churn_table_1.loc[comb, :]
    if chi2_contingency(subtable, correction=False)[1] < 0.05:
        n_pvals +=1
n_pvals

34

## Task 2
Какие проблемы Вы видите в построении анализа из первого вопроса? Отметьте все верные утверждения.

1. Анализ нужно было начинать с применения xи-квадрат к таблице сопряженности, в которой присутствовали сразу все возможные штаты. Достигаемой уровень значимости такой проверки = 0.7, что дает нам гарантию, что нет ни одной пары штатов, в которых отличие в соотношениях ушедших и оставшихся клиентов статистически значимо.
2. Применение критерия xи-квадрат для этих данных не обосновано, потому что не выполняются условия, при которых этот критерий дает правильные результаты.
3. Хи-квадрат используется для того, чтобы сравнить выборку с некоторым воздействием (treatment) и выборку без этого воздействия (control). Мы же в первом задании сравнивали штаты, используя данные только control группы. Для данных только из control группы использование xи-квадрат неправомерно.
4. Интерпретация числа достигаемых уровней значимости, меньших $\alpha=0.05$, некорректна, поскольку не сделана поправка на множественную проверку гипотез.
5. Поправку на множественную проверку здесь применять нельзя — она используется только для группы критериев, проверяющих равенство средних (типа t-критерия). Критерий xи-квадрат не принадлежит этому семейству, поэтому поправка не нужна.

In [8]:
chi2_contingency(state_churn_table_1)[1]

0.7097590042778473

In [9]:
(state_churn_table_1 < 5).values.sum() / (state_churn_table_1.shape[0] * state_churn_table_1.shape[1])

0.3333333333333333

**Ответ:** 2, 4.

Применение критерия хи-квадрат не обосновано, поскольку у нас больше 20% значений, которые меньше 5.

## Task 3
В основе критерия xи-квадрат лежит предположение о том, что если верна нулевая гипотеза, то дискретное биномиальное распределение данных по клеткам в таблице сопряженности может быть аппроксимировано с помощью непрерывного распределения xи-квадрат. Однако точность такой аппроксимации существенно зависит от суммарного количества наблюдений и их распределения в этой таблице (отсюда и ограничения при использовании критерия xи-квадрат).

Одним из способов коррекции точности аппроксимации является поправка Йетса на непрерывность. Эта поправка заключается в вычитании константы 0.5 из каждого модуля разности наблюденного $O_i$ и ожидаемого $E_i$ значений, то есть, статистика с такой поправкой выглядит так:

$$\chi _{\text{Yates}}^{2}=\sum _{i=1}^{N}{(|O_{i}-E_{i}|-0.5)^{2} \over E_{i}}.$$

Такая поправка, как несложно догадаться по формуле, как правило, уменьшает значение статистики $\chi^{2}$, то есть увеличивает достигаемый уровень значимости.

Эта поправка обычно используется для таблиц сопряженности размером 2x2 и для небольшого количества наблюдений. Такая поправка, однако, не является серебрянной пулей, и часто критикуется за то, что статистический критерий при ее использовании становится слишком консервативным, то есть часто не отвергает нулевую гипотезу там, где она неверна (совершает ошибку II рода).

Полезно знать, что эта поправка часто включена по умолчанию (например, в функции **scipy.stats.chi2_contingency**) и понимать ее влияние на оценку достигаемого уровня значимости.

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией и сравните полученные результаты, отметив все верные варианты.

1. Количество достигаемых уровней значимости, меньших, чем 0.05, в точности равно нулю. То есть поправка увеличила достигаемые уровни значимости настолько, что больше ни одно из значений достигаемого уровня значимости не попадает в диапазон от 0 до 0.05.
2. Поправка Йетса на непрерывность всегда увеличивает значение достигаемого уровня значимости, поэтому все получившиеся значения достигаемого уровня значимости строго больше или равны таковым значениям при отсутствии этой поправки.
3. Количество достигаемых уровней значимости, меньших, чем 0.05, почти не изменилось, нельзя сказать, что введенная поправка сильно поменяла достигаемые уровни значимости.
4. Достигаемые уровни значимости на наших данных, полученные с помощью критерия xи-квадрат с поправкой Йетса, в среднем получаются больше, чем соответствующие значения без поправки.

In [10]:
n_pvals_corr = 0
for comb in combinations(state_churn_table_1.index, 2):
    subtable = state_churn_table_1.loc[comb, :]
    if chi2_contingency(subtable, correction=True)[1] < 0.05:
        n_pvals_corr +=1
n_pvals_corr

0

In [11]:
np.all([chi2_contingency(state_churn_table_1.loc[comb, :], correction=True)[1] >= 
 chi2_contingency(state_churn_table_1.loc[comb, :], correction=False)[1]  
 for comb in combinations(state_churn_table_1.index, 2)])

False

In [12]:
print('Correction=True Mean:', np.mean([chi2_contingency(state_churn_table_1.loc[comb, :], correction=True)[1] 
                   for comb in combinations(state_churn_table_1.index, 2)]))
print('Correction=False Mean:', np.mean([chi2_contingency(state_churn_table_1.loc[comb, :], correction=False)[1] 
                   for comb in combinations(state_churn_table_1.index, 2)]))


Correction=True Mean: 0.6640566382051047
Correction=False Mean: 0.5018273798739158


**Ответ:** 1, 4

## Task 4
Что если у нас мало данных, мы не хотим использовать аппроксимацию дискретного распределения непрерывным и использовать сомнительную поправку, предположения критерия xи-квадрат не выполняются, а проверить гипотезу о том, что данные принадлежат одному распределению, нужно ?

В таком случае прибегают к так называемому точному критерию Фишера. Этот критерий не использует приближений и в точности вычисляет значение достигаемого уровня значимости используя комбинаторный подход.

Пусть у нас есть таблица сопряженности 2x2:

|   |Группа 1	|Группа 2	|$\Sigma$|
|-----------|:---------:|-------:|
|Воздействие 1|	$a$	|$b$	|$a+b$|
|Воздействие 2|	$c$	|$d$	|$c+d$|
|$\Sigma$	|$a + c$|	$b + d$| $n = a + b + c + d$|

Тогда вероятность получить именно такие $a, b, c, d$ при фиксированных значениях сумм по строкам и по столбцам) задается выражением:

$$p={\frac {\displaystyle {{a+b} \choose {a}}\displaystyle {{c+d} \choose {c}}}{\displaystyle {{n} \choose {a+c}}}}={ \dfrac {(a+b)!~(c+d)!~(a+c)!~(b+d)!}{a!~~b!~~c!~~d!~~n!}}$$. 

В числителе этой дроби стоит суммарное количество способов выбрать $a$ и $c$ из $a+b$ и $c+d$ соответственно. А в знаменателе — количество способов выбрать число объектов, равное сумме элементов первого столбца $a+c$ из общего количества рассматриваемых объектов $n$.

Чтобы посчитать достигаемый уровень значимости критерия Фишера, нужно перебрать все возможные значения $a, b, c, d$ в клетках этой таблицы так, чтобы построковые и постолбцовые суммы не изменились. Для каждого такого набора $a,b,c,d$ нужно вычислить значение $p_i$ по формуле выше и просуммировать все такие значения $p_i$, которые меньше или равны $p$, которое мы вычислили по наблюдаемым значениям $a,b,c,d$.

Понятно, что такой критерий вычислительно неудобен в силу большого количества факториалов в формуле выше. То есть даже при небольших выборках для вычисления значения этого критерия приходится оперировать очень большими числами. Поэтому данным критерием пользуются обычно только для таблиц 2x2, но сам критерий никак не ограничен количеством строк и столбцов, и его можно построить для любой таблицы $n\times m$.

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

Точный критерий Фишера удобно вычислять с помощью функции **scipy.stats.fisher_exact**, которая принимает на вход таблицу сопряженности 2x2.

1. Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат без поправки
2. Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем значительно большие, чем xи-квадрат без поправки
3. Точный критерий Фишера всегда лучше, чем критерий xи-квадрат, потому что не использует аппроксимацию дискретного распределения непрерывным. Однако при увеличении размера выборки его преимущества по сравнению с критерем xи-квадрат уменьшаются, в пределе достигая нуля.
4. Точный критерий Фишера точно также, как и критерий xи-квадрат, нельзя использовать, если наблюдений < 40 и если ожидаемое значение меньше 5 больше чем в 20% ячейках.
5. Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем большие, чем xи-квадрат с поправкой Йетса
6. Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат с поправкой Йетса

In [13]:
from scipy.stats import fisher_exact

In [14]:
print('Fisher test mean:', np.mean([fisher_exact(state_churn_table_1.loc[comb, :])[1] 
                   for comb in combinations(state_churn_table_1.index, 2)]))

Fisher test mean: 0.6483383060020681


**Ответ:** 2, 3, 6

## Task 5
Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике.

Рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость.

Отметьте все верные утверждения.

1. Все варианты неверны, потому что значимость корреляции Пирсона можно оценивать только для нормального распределения, как и упоминалось в лекциях.
2. Корреляция Пирсона имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.
3. Корреляция Пирсона имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.
4. Корреляция Пирсона имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.
5. Корреляция Пирсона имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.

In [19]:
from scipy.stats import pearsonr

In [20]:
print('r = {:.3f}, p_value = {:.3f}'.format(*pearsonr(data.day_calls, data.mes_estim)))

r = -0.052, p_value = 0.003


**Ответ:** 4

## Task 6
Еще раз рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость.

Отметьте все верные утверждения.

1. Корреляция Спирмена имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.
2. Корреляция Спирмена имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.
3. Корреляция Спирмена имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.
4. Корреляция Спирмена имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.
5. Корреляция Спирмена тут неприменима, поскольку речь идет о непрерывных величинах, а корреляция Спирмена применяется к выборочным рангам двух выборок.

In [15]:
from scipy.stats import spearmanr

In [17]:
print('r = {:.3f}, p_value = {:.3f}'.format(*spearmanr(data.day_calls, data.mes_estim)))

r = 0.043, p_value = 0.012


**Ответ:** 1

## Task 7 
Как можно интерпретировать полученные значения коэффициентов корреляции и достигаемые уровни значимости при проверки гипотез о равенстве нулю этих коэффициентов?

1. Не стоит ориентироваться на значение корреляции Спирмена, потому что корреляцию Спирмена можно считать только тогда, когда оба признака дискретные и между значениями можно установить строгий порядок.
2. Предположение нормальности данных двух признаков не выполнено, что хорошо видно на ку-ку графике, поэтому корреляция Пирсона может быть полностью неадекватна.
3. Посчитанные корреляции и их значимости говорят лишь о том, что необходимо взглянуть на данные глазами и попытаться понять, что приводит к таким (противоречивым?) результатам.
4. Подсчет корреляций не имеет особого смысла, поскольку корреляция ничего не говорит о том, какая на самом деле зависимость имеется между признаками.

**Ответ:** 3

## Task 8
Посчитайте значение коэффицента корреляции Крамера между двумя признаками: штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

1. Достигаемый уровень значимости < 0.05, то есть, отличие от нуля значения коэффицента Крамера значимо.
2. Достигаемый уровень значимости > 0.05, то есть, отличие от нуля значения коэффицента Крамера незначимо.
3. Для вычисления коэффициента Крамера используется значение статистики xи-квадрат, на которую мы не можем положиться применительно к нашим данным.
4. Коэффициент корреляции Крамера не может быть использован для сравнения связи этих двух признаков, потому что он используется для таблиц сопряженности, где каждая из размерностей больше двух. Если хотя бы одна из размерностей равна 2, то нужно использовать коэффициент корреляции Мэтьюса.

In [171]:
chi2, p, _, _ = chi2_contingency(state_churn_table_1)
print(f'chi2 = {chi2:.3f}, p_value = {p:.3f}')

chi2 = 44.053, p_value = 0.710


In [172]:
n = state_churn_table_1.values.sum()
print('V Cramer coeff: %.4f' % np.sqrt(chi2 / (n*(min(state_churn_table_1.shape)-1))))

V Cramer coeff: 0.2004


**Ответ:** 3

## Task 9
Вы прослушали большой курс и к текущему моменту обладете достаточными знаниями, чтобы попытаться самостоятельно выбрать нужный метод / инструмент / статистический критерий и сделать правильное заключение.

В этой части задания вам нужно будет самостоятельно решить, с помощью каких методов можно провести анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1).

Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

Не забудьте про поправку на множественную проверку! И не пользуйтесь односторонними альтернативами, поскольку вы не знаете, к каким действительно последствиям приводят тестируемые методы (treatment = 0, treatment = 2) !

1. Ни один из методов не показал значительного улучшения относительно других, о чем говорит групповой статистический критерий.
2. treatment = 0 статистически значимо отличается от контрольной группы treatment = 1
3. treatment = 2 статистически значимо отличается от контрольной группы treatment = 1
4. В дальнейшем телеком компании рекомендуется использовать и treatment = 0, и treatment = 2 для наибольшей эффективности удержения абонентов.
5. Отличие между treatment = 0 и treatment = 2 относительно влияния на уровень churn статистически незначимо.

In [109]:
state_churn_table_0 = pd.crosstab(data[data.treatment == 0].state, data[data.treatment == 0].churn)
state_churn_table_2 = pd.crosstab(data[data.treatment == 2].state, data[data.treatment == 2].churn)

In [110]:
state_churn_table_0.head()

churn,False,True
state,,
AK,15,2
AL,22,1
AR,14,3
AZ,26,0
CA,10,3


In [112]:
props_churn = []
props_churn.append(state_churn_table_0[True].sum() / state_churn_table_0.values.sum())
props_churn.append(state_churn_table_1[True].sum() / state_churn_table_1.values.sum())
props_churn.append(state_churn_table_2[True].sum() / state_churn_table_2.values.sum())
for i, prop in enumerate(props_churn):
    print(f'Treatment #{i} prop. of churn = {prop:.3f}')

Treatment #0 prop. of churn = 0.146
Treatment #1 prop. of churn = 0.164
Treatment #2 prop. of churn = 0.125


In [113]:
from statsmodels.stats.proportion import proportion_confint

In [116]:
for i, table in enumerate([state_churn_table_0, state_churn_table_1, state_churn_table_2]):
    left, right = proportion_confint(table[True].sum(), table.values.sum(), method='wilson')
    print(f'Treatment #{i} Conf. int. for prop. of churn = ({left:.4f}, {right:.4f})')

Treatment #0 Conf. int. for prop. of churn = (0.1263, 0.1674)
Treatment #1 Conf. int. for prop. of churn = (0.1433, 0.1872)
Treatment #2 Conf. int. for prop. of churn = (0.1069, 0.1459)


In [121]:
from scipy import stats

In [122]:
def proportions_diff_z_stat_ind(sample1, sample2):
    l1 = len(sample1)
    l2 = len(sample2)
    p1 = sample1.sum() / l1
    p2 = sample2.sum() / l2
    P = (p1*l1 + p2*l2) / (l1 + l2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / l1 + 1. / l2))

In [123]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [174]:
p_vals = []
for treat1, treat2 in [(1, 0), (1, 2), (0, 2)]:
    z_stat = proportions_diff_z_stat_ind(data[data.treatment == treat1].churn, data[data.treatment == treat2].churn)
    p_value = proportions_diff_z_test(z_stat)
    p_vals.append(p_value)
    print(f'Treatment #{treat1} and #{treat2}: p-value = {p_value:.3f}')

Treatment #1 and #0: p-value = 0.228
Treatment #1 and #2: p-value = 0.009
Treatment #0 and #2: p-value = 0.156


In [175]:
from statsmodels.stats.multitest import multipletests

In [176]:
multipletests(p_vals, method='holm')

(array([False,  True, False]),
 array([0.31284938, 0.02804425, 0.31284938]),
 0.016952427508441503,
 0.016666666666666666)

**Ответ:** 3, 5